In [4]:
import csv
from datetime import datetime

def leer_csv(archivo):
    with open(archivo, 'r') as file:
        reader = csv.reader(file)
        data = list(reader)
    return data

def filtrar_datos(datos):
    datos_filtrados = []
    for row in datos:
        ci, nombre1, nombre2, paterno, materno, fecha = row
        if not any(char.isdigit() for char in (nombre1 + nombre2 + paterno + materno)) and not any(char.isalpha() for char in ci):
            datos_filtrados.append((ci, nombre1, nombre2, paterno, materno, fecha))
    return datos_filtrados

def unir_nombres(datos_filtrados):
    datos_unidos = []
    for ci, nombre1, nombre2, paterno, materno, fecha in datos_filtrados:
        nombre_completo = f"{nombre1} {nombre2} {paterno} {materno}"
        datos_unidos.append((ci, nombre_completo, fecha))
    return datos_unidos

def convertir_fecha(datos_unidos):
    datos_final = []
    for ci, nombre, fecha in datos_unidos:
        fecha = datetime.strptime(fecha, '%d/%m/%Y')
        fecha = fecha.strftime('%Y-%m-%d')
        datos_final.append((ci, nombre, fecha))
    return datos_final

def escribir_csv(datos_final):
    with open('datos_final.csv', 'w') as file:
        writer = csv.writer(file)
        writer.writerows(datos_final)

In [5]:
archivo_csv = "ciudadanos - ciudadanos.csv"
datos = leer_csv(archivo_csv)
datos[2:3]

[['3836432TF', 'JUAN', 'LINO', 'LURICI', 'ROCA', '06/05/1959']]

In [6]:
datos_filtrados = filtrar_datos(datos)
datos_filtrados[2:3]

[('6115816', 'SUANY', '', 'ROA', 'SANGUINO', '25/09/1986')]

In [7]:
nombres = unir_nombres(datos_filtrados)
nombres[2:3]

[('6115816', 'SUANY  ROA SANGUINO', '25/09/1986')]

In [8]:
fecha = convertir_fecha(nombres)
fecha[2:3]

[('6115816', 'SUANY  ROA SANGUINO', '1986-09-25')]

In [9]:
escribir_csv(fecha)

In [19]:
import mysql.connector
import csv

def agregar_bd(ruta_csv):
    try:
        # Establece la conexión a la base de datos MySQL
        conexion = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="bd_personas"
        )

        # Crea un objeto cursor para ejecutar consultas SQL
        cursor = conexion.cursor()
        # Abrir el archivo CSV y leer los datos
        with open(ruta_csv, 'r') as csv_file:
            lector_csv = csv.reader(csv_file)
            # Iterar a través de las filas en el archivo CSV y agregarlas a la base de datos
            for fila in lector_csv:
                ci, nombres, fecha = fila
                # Consulta para insertar datos en la tabla de la base de datos
                consulta = "INSERT INTO ciudadano (ci, nombre, fecha) VALUES (%s, %s, %s)"
                valores = (ci, nombres, fecha)
                # Ejecutar la consulta
                cursor.execute(consulta, valores)

        # Confirmar la transacción y cerrar la conexión
        conexion.commit()
        cursor.close()
        conexion.close()
        print("Datos agregados correctamente a la base de datos.")
    except Exception as error:
        print(f"Error al agregar datos a la base de datos: {error}")

In [21]:
agregar_bd("datos_final.csv")

Datos agregados correctamente a la base de datos.
